In [1]:
loadPackage "Polyhedra"


o1 = Polyhedra

o1 : Package


## Get the Wasserstein Ball under the Discrete Metric

In [3]:
R = QQ
n = 3


o3 = 3


In [4]:
A = mutableMatrix(R,n*(n-1),n)


o4 = 0

o4 : MutableMatrix


In [7]:
k = 0
for i from 0 to (n - 1) do (
    for j from 0 to (n - 1) do (
        if i < j then (
            A_(k,i) = 1;
            A_(k,j) = -1;
            k = k + 1;
            A_(k,i) = -1;
            A_(k,j) = 1;
            k = k + 1;
        )
    )
)

A = matrix A



o7 = | 1  -1 0  |
     | -1 1  0  |
     | 1  0  -1 |
     | -1 0  1  |
     | 0  1  -1 |
     | 0  -1 1  |

              6        3
o7 : Matrix QQ  <--- QQ


In [8]:
A


o8 = | 1  -1 0  |
     | -1 1  0  |
     | 1  0  -1 |
     | -1 0  1  |
     | 0  1  -1 |
     | 0  -1 1  |

              6        3
o8 : Matrix QQ  <--- QQ


In [11]:
v = mutableMatrix(R,n*(n-1),1)

for i from 0 to (n*(n-1) - 1) do (
    v_(i,0) = 1
)

v = matrix v



o11 = | 1 |
      | 1 |
      | 1 |
      | 1 |
      | 1 |
      | 1 |

               6        1
o11 : Matrix QQ  <--- QQ


In [14]:
c = mutableMatrix(R,1,n)

for i from 0 to (n - 1) do (
    c_(0,i) = 1
)

c = matrix c



o14 = | 1 1 1 |

               1        3
o14 : Matrix QQ  <--- QQ


In [15]:
w = matrix {{0}}


o15 = 0

               1        1
o15 : Matrix ZZ  <--- ZZ


In [16]:
myPolyhedron = polyhedronFromHData(A, v, c, w)


o16 = myPolyhedron

o16 : Polyhedron


In [17]:
V = vertices myPolyhedron


o17 = | 1/3  2/3  -1/3 1/3  -2/3 -1/3 |
      | 1/3  -1/3 2/3  -2/3 1/3  -1/3 |
      | -2/3 -1/3 -1/3 1/3  1/3  2/3  |

               3        6
o17 : Matrix QQ  <--- QQ


In [21]:
numVertices = (fVector myPolyhedron)_0

B = mutableMatrix(R,numVertices,n)

for i from 0 to (numVertices - 1) do (
    currVec = transpose matrix V_(i);
    for j from 0 to (n-1) do (
        B_(i,j) = currVec_(0,j)
    )
)

B = matrix B



o21 = | 1/3  1/3  -2/3 |
      | 2/3  -1/3 -1/3 |
      | -1/3 2/3  -1/3 |
      | 1/3  -2/3 1/3  |
      | -2/3 1/3  1/3  |
      | -1/3 -1/3 2/3  |

               6        3
o21 : Matrix QQ  <--- QQ


In [24]:
v = mutableMatrix(R,numVertices,1)

for i from 0 to (numVertices - 1) do (
    v_(i,0) = 1
)

v = matrix v



o24 = | 1 |
      | 1 |
      | 1 |
      | 1 |
      | 1 |
      | 1 |

               6        1
o24 : Matrix QQ  <--- QQ


In [26]:
PDual = polyhedronFromHData(B, v, c, w)
vertPDual = vertices PDual


o26 = | 1  -1 1  0  -1 0  |
      | -1 1  0  1  0  -1 |
      | 0  0  -1 -1 1  1  |

               3        6
o26 : Matrix QQ  <--- QQ


## Define the Twisted Quintic within a ring $R$

In [34]:
R = QQ[p0, p1, p2];
P = {p0, p1, p2};

definingEqns = matrix{{p1^2-4*p0*p2}}

G = ideal(definingEqns);
GGens = gens G;

mu = matrix{{1/2,1/4,1/4}};
pTranslated = matrix{{p0, p1, p2}};
pTranslated = pTranslated - mu;




o30 : Ideal of R


              1       1
o31 : Matrix R  <--- R


               1        3
o32 : Matrix QQ  <--- QQ


              1       3
o33 : Matrix R  <--- R


              1       3
o34 : Matrix R  <--- R


In [37]:
diffConstraintList = {};

for i from 0 to (numgens(G)-1) do (
    L = {};
    for j from 0 to (#P-1) do (
        L = append(L, diff(P_j,GGens_i))
    );
    diffConstraintList = append(diffConstraintList, L);
);

diffConstraintList = transpose matrix diffConstraintList;




              3       1
o37 : Matrix R  <--- R


## For Codimension 1

In [43]:
codimension = 1;
L = faces(codimension,PDual);

listOfAmbientVertices = apply(L, f -> vertPDual_(f#0));

B = apply(listOfAmbientVertices, A -> generators(ker transpose matrix A));

-- Get the normals to all of the faces
currNormals = {};
for i from 0 to (#listOfAmbientVertices - 1) do (
    
    ambientVertices = listOfAmbientVertices#i;
    
    currNormal = mutableMatrix(R,1,n);
    currNormal = mutableMatrix currNormal;
        
    for j from 0 to (n - 1) do (
        for colIndex from 0 to (numcols(ambientVertices) - 1) do (
            currNormal_(0,j) = (currNormal_(0,j) + ambientVertices_(j,colIndex))
        );
    );
    
    for j from 0 to (n - 1) do (
        currNormal_(0,j) = (currNormal_(0,j) / (numcols ambientVertices))
    );
    
    currNormals = append(currNormals, matrix currNormal)
);

In [46]:
-- Generate D Matrix and Intersection Ideals for every set of ambient verticies that form a facet of the desired codimension
DLists = {}
intersectionIdealLists = {}

for i from 0 to (#listOfAmbientVertices - 1) do (

    D = {};
    D = append(D, currNormals_i);
                
    for j from 0 to (numcols(diffConstraintList) - 1) do (
        D = append(D, transpose matrix {diffConstraintList_j} );
    );
    
    D = append(D, c);
    
    intersectionIdeal = {};

    Q = B_i;
    for j from 0 to (numcols(Q) - 1) do (
        currVec = transpose matrix Q_j;
        D = append(D, currVec);
        dummyVec = currVec * transpose(pTranslated);
        intersectionIdeal = append(intersectionIdeal, dummyVec_0 );
    );
    
    for i from 0 to (numgens(G)-1) do (
        intersectionIdeal = append(intersectionIdeal, GGens_i);
    );
    
    DLists = append(DLists, D);
    intersectionIdealLists = append(intersectionIdealLists, intersectionIdeal);
);

In [48]:
-- purely for converting a list of vectors into a list of matrices
convertDListsToMatrices = {}
for i from 0 to (#listOfAmbientVertices - 1) do (
    D = DLists_i;
    DMatrix = mutableMatrix(R,#D,n);
    for j from 0 to (#D - 1) do (
        currVec = D_j;
        for k from 0 to (n-1) do (
            dummyVec = currVec_k;
            DMatrix_(j,k) = dummyVec_0;
        );
    );
    DMatrix = matrix DMatrix;
    convertDListsToMatrices = append(convertDListsToMatrices, DMatrix)
);

In [50]:
-- purely for converting a list of vectors of functions into an ideal
convertintersectionIdealListsToIdeals = {}
for i from 0 to (#listOfAmbientVertices - 1) do (
    intersectionIdeal = intersectionIdealLists_i;
    myLocalList = {};
    for j from 0 to (#intersectionIdeal - 1) do (
        dummyVec = intersectionIdeal_j;
        myLocalList = append(myLocalList, dummyVec_0);
    );
    convertintersectionIdealListsToIdeals = append(convertintersectionIdealListsToIdeals, ideal myLocalList);
);

In [54]:
exampleIndex = 3
print(listOfAmbientVertices_exampleIndex)
print(convertintersectionIdealListsToIdeals_exampleIndex)
print(convertDListsToMatrices_exampleIndex)

| 1  1  |
| -1 0  |
| 0  -1 |

                           2
ideal (p0 + p1 + p2 - 1, p1  - 4p0*p2)

| 1    -1/2 -1/2 |
| -4p2 2p1  -4p0 |
| 1    1    1    |
| 1    1    1    |


In [55]:
for i from 0 to (#listOfAmbientVertices - 1) do (
    I = convertintersectionIdealListsToIdeals_i;
    D = convertDListsToMatrices_i;
    Minors = minors(n, D);
    J = I + Minors;
    print(degree J);
);

1
1
1
1
1
1


## For Codimension 2

In [61]:
codimension = 2;
L = faces(codimension,PDual);

listOfAmbientVertices = apply(L, f -> vertPDual_(f#0));

B = apply(listOfAmbientVertices, A -> generators(ker transpose matrix A));

-- Get the normals to all of the faces
currNormals = {};
for i from 0 to (#listOfAmbientVertices - 1) do (
    
    ambientVertices = listOfAmbientVertices#i;
    
    currNormal = mutableMatrix(R,1,n);
    currNormal = mutableMatrix currNormal;
        
    for j from 0 to (n - 1) do (
        for colIndex from 0 to (numcols(ambientVertices) - 1) do (
            currNormal_(0,j) = (currNormal_(0,j) + ambientVertices_(j,colIndex))
        );
    );
    
    for j from 0 to (n - 1) do (
        currNormal_(0,j) = (currNormal_(0,j) / (numcols ambientVertices))
    );
    
    currNormals = append(currNormals, matrix currNormal)
);

In [63]:
-- purely for converting a list of vectors of functions into an ideal
convertintersectionIdealListsToIdeals = {}
for i from 0 to (#listOfAmbientVertices - 1) do (
    intersectionIdeal = intersectionIdealLists_i;
    myLocalList = {};
    for j from 0 to (#intersectionIdeal - 1) do (
        dummyVec = intersectionIdeal_j;
        myLocalList = append(myLocalList, dummyVec_0);
    );
    convertintersectionIdealListsToIdeals = append(convertintersectionIdealListsToIdeals, ideal myLocalList);
);

In [64]:
for i from 0 to (#listOfAmbientVertices - 1) do (
    I = convertintersectionIdealListsToIdeals_i;
    J = I;
    print(degree J);
);

2
2
2
2
2
2
